In [ ]:
import psycopg2
from psycopg2 import pool
import psycopg2.extras
import pandas as pd
from functools import wraps
from shapely import wkb
from shapely import wkt
import threading
import copy
import json
from psycopg2.pool import ThreadedConnectionPool
from tqdm import tqdm



connection_pool = None
lock = threading.Lock()

def init_connection_pool():
    global connection_pool

    connection_pool = ThreadedConnectionPool(
        10, 100,
        dbname="harbin_platform",
        user='postgres',
        password='Map12345',
        host='121.43.234.148',
        port='5432'
    )

def get_connection():
    if connection_pool is None:
        raise ValueError("Connection pool has not been initialized. Call init_connection_pool(city_name) first.")
    with lock:
        return connection_pool.getconn()

def release_connection(conn):
    connection_pool.putconn(conn)

def with_connection(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        conn = get_connection()
        conn.set_client_encoding('UTF8')
        psycopg2.extras.register_hstore(conn)
        cur = conn.cursor()
        try:
            result = func(cur, *args, **kwargs)
            conn.commit()
        finally:
            cur.close()
            release_connection(conn)
        return result

    return wrapper

In [ ]:
from shapely import wkt
from tqdm import tqdm
import json

@with_connection
def fetch_all_trips(cur):
    cur.execute("SELECT id, road_ids FROM trips")
    return cur.fetchall()

@with_connection
def fetch_all_geoms(cur, gid_list):
    sql = "SELECT gid, ST_AsText(geom) FROM bfmap_ways WHERE gid = ANY(%s)"
    cur.execute(sql, (gid_list,))
    return dict(cur.fetchall())

@with_connection
def bulk_update_trip_coords(cur, id_coords_pairs):
    # 构造 VALUES 语句： [(id1, json1), (id2, json2), ...]
    psycopg2.extras.execute_values(
        cur,
        """
        UPDATE trips AS t SET coord_list = v.coord_list
        FROM (VALUES %s) AS v(id, coord_list)
        WHERE t.id = v.id
        """,
        [(tid, json.dumps(coords)) for tid, coords in id_coords_pairs],
        template="(%s::bigint, %s::jsonb)"
    )

def linestring_to_coords(linestr):
    line = wkt.loads(linestr)
    return [[pt[0], pt[1]] for pt in line.coords]

def process_some_trips():
    print(f"Step 1: Fetching all trips")
    trips = fetch_all_trips()

    print("Step 2: Collecting all unique GIDs")
    all_gids = set()
    for _, road_ids in trips:
        if isinstance(road_ids, list):
            all_gids.update(road_ids)

    print(f"Step 3: Fetching {len(all_gids)} geometries in bulk")
    gid2geom = fetch_all_geoms(list(all_gids))

    print("Step 4: Building update payload")
    update_payload = []
    for trip_id, road_ids in tqdm(trips, desc="Processing"):
        if not isinstance(road_ids, list) or len(road_ids) == 0:
            continue
        full_coords = []
        for i, gid in enumerate(road_ids):
            linestr = gid2geom.get(gid)
            if not linestr:
                continue
            coords = linestring_to_coords(linestr)
            if i > 0 and full_coords and coords and coords[0] == full_coords[-1]:
                coords = coords[1:]
            full_coords.extend(coords)
        update_payload.append((trip_id, full_coords))

    print("Step 5: Writing all coords back at once")
    bulk_update_trip_coords(update_payload)

# 执行
init_connection_pool()
process_some_trips()

In [2]:
from django.contrib.gis import gdal
print(gdal.VersionInfo('release_name'))


ImproperlyConfigured: Could not find the GDAL library (tried "gdal", "GDAL", "gdal3.6.0", "gdal3.5.0", "gdal3.4.0", "gdal3.3.0", "gdal3.2.0", "gdal3.1.0", "gdal3.0.0", "gdal2.4.0", "gdal2.3.0", "gdal2.2.0"). Is GDAL installed? If it is, try setting GDAL_LIBRARY_PATH in your settings.